## For Scraping of Lazada Review

### 1. Import packages


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time
import math
from datetime import datetime

### 2. Initalise Columns

In [3]:
brand_csv=[]
category_csv =[]
titles_csv = []
prices_csv = []
reviewer_csv=[]
review_csv=[]
product_csv = []
rating_csv =[]
date_review_csv = []
response_csv=[]

In [2]:
print(datetime.now())

2020-03-23 09:19:20.856564


### 3. Begin the Scraping Process For Each Loreal Brands

In [4]:
#Open Chrome
driver = webdriver.Chrome(executable_path='chromedriver')

brands = ['L-oreal-Paris','Garnier', 'Maybelline']
for a in range(len(brands)):
    #open the link to the respective brands All Products page
    driver.get('https://www.lazada.sg/'+brands[a]+'-official-store/?langFlag=en&q=All-Products&from=wangpu&pageTypeId=2')
   
    brand_name = brands[a]
    
    #Find the categories placeholder at the left side
    categories = driver.find_elements_by_class_name('cds557')

    main_cat =[]
    titles = []
    prices = []
    urls = []   
    category_type=[]
    
    for category in categories:
        main_cat_link = category.get_attribute('href')
        main_cat.append(main_cat_link)

    for ii in range(len(main_cat)):
        #open the urls to get all the products in the categories page
        driver.get(main_cat[ii])
        category = driver.find_element_by_class_name('c2BJaq').text
        print("Category:" + category)
        products = driver.find_elements_by_class_name('c2prKC')
        
        for product in products:
            category_type.append(category)
            titles.append(product.find_element_by_class_name('c16H9d').text)
            prices.append(product.find_element_by_class_name('c3gUW0').text)

            #retrieve each product's link
            link = product.find_element_by_class_name('c16H9d').find_element_by_css_selector('a').get_attribute('href')
            urls.append(link)
        #print(urls)
        
    #retrieve each product's reviews
    for i in range(len(urls)):
        time.sleep(10)
        driver.implicitly_wait(5)
        count_total =0
        driver.get(urls[i])
        print('Progess:',i+1,'/',len(titles))
        print(titles[i])

        timeout = 15
        try:
            WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CLASS_NAME, "pdp-block")))
        except TimeoutException:
            pass
        try:
            total_pages =1
            pages = driver.find_elements_by_css_selector("[class='next-btn next-btn-normal next-btn-medium next-pagination-item']")
            total_pages = int(pages[-1].text)
        except:
            pass

        for j in range(int(total_pages)):
            #Get the review details here
            time.sleep(2)
            total = 0
            try:
                reviews_area = driver.find_elements_by_css_selector("[class='mod-reviews']")
                try:
                    product_reviews = driver.find_elements_by_css_selector("[class='item']")
                except NoSuchElementException:
                    break

                # Get the review from each product
                for product_review in product_reviews:
                    time.sleep(2)
                    
                    # Star rating
                    star_ratings = product_review.find_elements_by_css_selector("[class='star']")
                    stars = "https://laz-img-cdn.alicdn.com/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

                    star_rate = 0
                    for rating in star_ratings:
                        if (rating.get_attribute('src') == stars):
                            star_rate = star_rate + 1
                    rating_csv.append(star_rate)
                    
                    #Reviewers
                    reviewer = product_review.find_element_by_css_selector("[class='middle']").text
                    reviewer = reviewer.replace('by', '')
                    reviewer = reviewer.replace('Verified Purchase', '')
                    reviewer_csv.append(reviewer)

                    # Date of Review
                    date = product_review.find_element_by_css_selector("[class='title right']").text
                    date_review_csv.append(date)

                    #Review
                    review = product_review.find_element_by_css_selector("[class='content']").text

                    if (len(review)>0 ):
                        review_csv.append(review)
                    else:
                        review_csv.append("No comments/review is an image")

                    time.sleep(2)
                    
                    #Brand
                    brand_csv.append(brand_name)
                    #Categories
                    category_csv.append(category_type[i])
                    #Title of the Product
                    titles_csv.append(titles[i])
                    #Price of the Product
                    prices_csv.append(prices[i])

                    # Product Purchase
                    # Check if the product purchase exists
                    try:
                        product_purchase = product_review.find_element_by_css_selector("[class='skuInfo']").text
                        product_csv.append(product_purchase)
                    except:
                        product_csv.append("Nil")

                    # Check if there is any  Customer Service Response
                    try:
                        response = product_review.find_element_by_css_selector("[class='item-content item-content--seller-reply']")
                        response_csv.append("yes")
                    except:
                        response_csv.append("no")

                    total +=1

                print("per page",total)
                count_total += total
                
                #Proceed to the next page
                if (int(total_pages))>1:
                    try:
                        time.sleep(5)
                        button_next=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.next-pagination-item.next")))
                        driver.execute_script("arguments[0].click();", button_next)
                        print("Next Page")
                    except:
                        break
            except StaleElementReferenceException:
                time.sleep(5)
                continue
        print("total review scrpaed : ",count_total)
        
driver.close()

Category:Makeup Removers
Category:Face Cleanser
Category:Facial Cleanser
Category:Skin Care Gifts & Value Sets
Category:Face Mask and Packs
Category:Face Scrubs & Exfoliators
Category:Dermacare Facial Moisturizers
Category:Facial Moisturizers
Category:Men's Moisturizers and Cream
Category:Toner & Mists
Category:Face Treatments Serums
Progess: 1 / 59
Micellar Cleansing Water Pink 400ml (For Sensitive Skin) by Garnier
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Ne

per page 0
total review scrpaed :  0
Progess: 45 / 59
[Essential Brigthening Regime] Garnier Light Day Cream 50ml + Super Essence 30ml
per page 0
total review scrpaed :  0
Progess: 46 / 59
Garnier Moisture Bomb Eye Mask - Hyaluronic Acid and Coconut Water
per page 3
total review scrpaed :  3
Progess: 47 / 59
[Bundle of 6] Garnier Hydra Bomb Serum Sheet Mask + 50ml Micellar Water
per page 0
total review scrpaed :  0
Progess: 48 / 59
[Bundle of 15] Garnier Hydra Bomb Serum Mask
per page 0
total review scrpaed :  0
Progess: 49 / 59
Garnier Light Complete Yogurt Sleeping Mask
per page 5
Next Page
per page 1
Next Page
total review scrpaed :  6
Progess: 50 / 59
Garnier Moisture Bomb Eye Mask - Hyaluronic Acid and Orange Juice
per page 5
Next Page
per page 4
Next Page
total review scrpaed :  9
Progess: 51 / 59
[Bundle of 5s] Garnier Pure Active Matcha Clay Mask Acne Care
per page 4
total review scrpaed :  4
Progess: 52 / 59
[Bundle of 2s] Garnier Light Brightening Scrub Wash 100ml
per page 1


per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 1
Next Page
total review scrpaed :  66
Progess: 35 / 50
Maybelline Master Camouflage Concealer Palette
per page 4
total review scrpaed :  4
Progess: 36 / 50
Maybelline Master Chrome Rose Gold Highlighter
per page 1
total review scrpaed :  1
Progess: 37 / 50
Maybelline Master Holographic Powder Highlighter 100
per page 3
total review scrpaed :  3
Progess: 38 / 50
Maybelline Facestudio Master Chrome Molten Gold Highlighter
per page 5
Next Page
per page 4
Next Page
total review scrpaed :  9
Progess: 39 / 50
Superstay Matte Ink Liquid Lipstick - All Ranges by Maybelline
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Page
per page 5
Next Pag

### 4.Save all the data into a file

In [6]:
#convert and save all data to csv file
platform = "Lazada"
data = {'Platform':platform ,'Brand': brand_csv, 'Category': category_csv, 'Product_Name ': titles_csv, 'Price':prices_csv,'Reviewer':reviewer_csv,'Review':review_csv,'Product_Purchase':product_csv,'Ratings':rating_csv,'Date_Of_Review':date_review_csv,'Response': response_csv}
df_loreal = pd.DataFrame.from_dict(data)
df_loreal.to_csv('Lazada_LorealAllProducts_23032020.csv')

In [18]:
## Merge the competitior and lazada daatset

#load in the competitors:
competitors_file = "CompetitorsAnalysis_18032020.csv"
competitors = pd.read_csv(loreal_file)

merge_file = pd.concat([df_loreal, competitors])
del merge_file['Unnamed: 0']
merge_file.to_csv('LazadaAllProducts.csv')